In [33]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List,Tuple, Set, Union, Dict
from pathlib import Path
from dataclasses import dataclass
from collections import defaultdict, OrderedDict
from IPython.display import display
np.set_printoptions(precision=3)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 59214,
  "iopub_port": 59215,
  "stdin_port": 59216,
  "control_port": 59217,
  "hb_port": 59218,
  "ip": "127.0.0.1",
  "key": "5905d580-665f9d96bcd048b2f3f37eb9",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-eb87ed9b-b0ea-439e-b3f7-cf67fc025009.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned ON


THE :  1.81        ERE :  0.31        HES :  0.24
AND :  0.73        TIO :  0.31        VER :  0.24
ING :  0.72        TER :  0.30        HIS :  0.24
ENT :  0.42        EST :  0.28        OFT :  0.22
ION :  0.42        ERS :  0.28        ITH :  0.21
HER :  0.36        ATI :  0.26        FTH :  0.21
FOR :  0.34        HAT :  0.26        STH :  0.21
THA :  0.33        ATE :  0.25        OTH :  0.21
NTH :  0.33        ALL :  0.25        RES :  0.21
INT :  0.32        ETH :  0.24        ONT :  0.20

TH :  2.71        EN :  1.13        NG :  0.89
HE :  2.33        AT :  1.12        AL :  0.88
IN :  2.03        ED :  1.08        IT :  0.88
ER :  1.78        ND :  1.07        AS :  0.87
AN :  1.61        TO :  1.07        IS :  0.86
RE :  1.41        OR :  1.06        HA :  0.83
ES :  1.32        EA :  1.00        ET :  0.76
ON :  1.32        TI :  0.99        SE :  0.73
ST :  1.25        AR :  0.98        OU :  0.72
NT :  1.17        TE :  0.98        OF :  0.71


http://practicalcryptography.com/cryptanalysis/letter-frequencies-various-languages/english-letter-frequencies/

In [42]:
def ngram_count_to_freqs(ngrams: Dict[str, int], total_count: int=None, 
                         percentage: bool=True, sort: bool=False):
    coef = 100.0 if percentage else 1.0
    if total_count is None:
        total_count = sum(ngrams.values())
    data = [(ngram, coef*count / total_count) for ngram, count in ngrams.items()]
    if sort:
        return OrderedDict(sorted(data, key=lambda x: x[1], reverse=True))
    return dict(data)
    
    
def load_ngrams(file: Union[str, Path], percentage: bool=True) -> Tuple[Dict[str, float], float]:
    file = Path(file)
    total_count = 0
    ngrams = {}
    coef = 100.0 if percentage else 1.0
    with file.open() as f:
        for line in f:
            ngram, count = line.strip().split()
            count = int(count)
            ngrams[ngram] = count
            total_count += count
    ngrams = ngram_count_to_freqs(ngrams, total_count=total_count, percentage=percentage, sort=True)
    return ngrams, list(ngrams.values())[-1]

In [109]:
trigrams, min_trigram = load_ngrams(r"E:\Source\Personal\University\2019\Crypto\data\english_trigrams.txt")
quadgrams, min_quadgram = load_ngrams(r"E:\Source\Personal\University\2019\Crypto\data\english_quadgrams.txt")
bigrams, min_bigram = load_ngrams(r"E:\Source\Personal\University\2019\Crypto\data\english_bigrams_1.txt")

In [187]:
# 1.6.12
crypto = "TNOSSKAIMAGAEITMHETHTSRH--IHEU-D-\
NUEIDSATDTDDSARAHHENTTTDSOUIOEART\
FHDAOMWYWFERTNEONFDYAHSEIMEDGRWTA\
TISURUARTHJ"

crypt_size = (10, 11)

def to_df(text, size):
    data = []
    for i in range(size[0]):
        row = []
        for j in range(size[1]):
            row.append(text[i*size[1]+j])
        data.append(row)
    return pd.DataFrame(data)

def from_df(matrix):
    texts = []
    for i in matrix.index:
        texts.append("".join([s for s in matrix.loc[i]]))
    return texts

def make_ngrams(text, n=2):
    ngrams = zip(*[text[i:] for i in range(n)])
    return ["".join(ngram) for ngram in ngrams]

def count_ngrams(text, n=2):
    counts = defaultdict(int)
    ngrams = make_ngrams(text, n=n)
    total_count = len(ngrams)
    for ngram in ngrams:
        counts[ngram] += 1
    return counts, total_count

def logloss(predicted, actual, prob_predicted, prob_actual):
    return np.abs(prob_predicted - prob_actual)

def ngram_fitness(matrix, n=3, loss=logloss, percentage=True) -> Tuple[dict, float]:
    text = "".join(from_df(matrix))
    logger.debug(f"Decoded text: {text}")
    ngram_counts, total_ngram_count = count_ngrams(text, n=n)
    ngram_freqs = ngram_count_to_freqs(ngram_counts, total_count=total_ngram_count, percentage=percentage)
    if n == 2:
        ngrams_natural, default_value = bigrams, min_bigram
    elif n == 3:
        ngrams_natural, default_value = trigrams, min_trigram
    elif n == 4:
        ngrams_natural, default_value = quadgrams, min_quadgram
    else:
        raise NotImplementedError(f"N gram of n={n} not supported")
    target = 0
    for (ngram_predicted, prob_predicted) in ngram_freqs.items():
        ngram_actual = ngram_predicted
        prob_actual = ngrams_natural.get(ngram_predicted, 0)
        #print(f"Predicted: {ngram_predicted} ({prob_predicted:.3f}), "
        #      f"actual: {ngram_actual} ({prob_actual:.3f})")
        target += loss(predicted=ngram_predicted, actual=ngram_actual, 
                       prob_predicted=prob_predicted, prob_actual=prob_actual)
        
    return ngram_freqs, target
    
df = to_df(crypto, crypt_size)
display(df)
ngram_fitness(df)

,0,1,2,3,4,5,6,7,8,9,10
0,T,N,O,S,S,K,A,I,M,A,G
1,A,E,I,T,M,H,E,T,H,T,S
2,R,H,-,-,I,H,E,U,-,D,-
3,N,U,E,I,D,S,A,T,D,T,D
4,D,S,A,R,A,H,H,E,N,T,T
5,T,D,S,O,U,I,O,E,A,R,T
6,F,H,D,A,O,M,W,Y,W,F,E
7,R,T,N,E,O,N,F,D,Y,A,H
8,S,E,I,M,E,D,G,R,W,T,A
9,T,I,S,U,R,U,A,R,T,H,J


({'TNO': 0.9259259259259259,
  'NOS': 0.9259259259259259,
  'OSS': 0.9259259259259259,
  'SSK': 0.9259259259259259,
  'SKA': 0.9259259259259259,
  'KAI': 0.9259259259259259,
  'AIM': 0.9259259259259259,
  'IMA': 0.9259259259259259,
  'MAG': 0.9259259259259259,
  'AGA': 0.9259259259259259,
  'GAE': 0.9259259259259259,
  'AEI': 0.9259259259259259,
  'EIT': 0.9259259259259259,
  'ITM': 0.9259259259259259,
  'TMH': 0.9259259259259259,
  'MHE': 0.9259259259259259,
  'HET': 0.9259259259259259,
  'ETH': 0.9259259259259259,
  'THT': 0.9259259259259259,
  'HTS': 0.9259259259259259,
  'TSR': 0.9259259259259259,
  'SRH': 0.9259259259259259,
  'RH-': 0.9259259259259259,
  'H--': 0.9259259259259259,
  '--I': 0.9259259259259259,
  '-IH': 0.9259259259259259,
  'IHE': 0.9259259259259259,
  'HEU': 0.9259259259259259,
  'EU-': 0.9259259259259259,
  'U-D': 0.9259259259259259,
  '-D-': 0.9259259259259259,
  'D-N': 0.9259259259259259,
  '-NU': 0.9259259259259259,
  'NUE': 0.9259259259259259,
  'UEI': 0.925

In [165]:
def swap(df, *swap_pairs, direction="columns"):
    indices = list(df.columns if direction == "columns" else df.index)
    def _swap(pair):
        idx1, idx2 = pair
        temp = indices[idx2]
        indices[idx2] = indices[idx1]
        indices[idx1] = temp
    for swap_pair in swap_pairs:
        _swap(swap_pair)
    return df.reindex(**{
        "columns" if direction == "columns" else "index": indices 
    })

def decode(matrix, k1=None, k2=None):
    if not k1 and not k2:
        logger.warning("No key was passed to decode, leaving as is")
        return matrix
    if k1:
        matrix = matrix.reindex(index=k1)
    if k2:
        matrix = matrix.reindex(columns=k2)
    return matrix

In [181]:
odf = swap(df, (5, 6), (2, 3), (1, 10), (5, 10))
display(odf)

,0,10,3,2,4,1,5,7,8,9,6
0,T,G,S,O,S,N,K,I,M,A,A
1,A,S,T,I,M,E,H,T,H,T,E
2,R,-,-,-,I,H,H,U,-,D,E
3,N,D,I,E,D,U,S,T,D,T,A
4,D,T,R,A,A,S,H,E,N,T,H
5,T,T,O,S,U,D,I,E,A,R,O
6,F,E,A,D,O,H,M,Y,W,F,W
7,R,H,E,N,O,T,N,D,Y,A,F
8,S,A,M,I,E,E,D,R,W,T,G
9,T,J,U,S,R,I,U,R,T,H,A


In [182]:
import itertools
import copy
keyspace = itertools.permutations(list(range(11)))

def swap_elements(a, swap: Tuple[int, int]):
    a = copy.copy(a)
    tmp = a[swap[0]]
    a[swap[0]] = a[swap[1]]
    a[swap[1]] = tmp
    return a

In [188]:
# OLD DO NOT USE
import random
max_iters = 100000
beginning = 100000
best_keys = []
best_loss = 0

for idx, key in enumerate(keyspace, -beginning):
    if idx < 0:
        continue
    logger.debug(f"[{idx+1}/{max_iters}] Trying key {key}...")
    if idx >= max_iters:
        logger.warning(f"[STOPPING!] Best key loss: {best_loss}, keys {len(best_keys)}")
        raise RuntimeError("Key not found")
    decoded_matrix = decode(df, k2=key)
    # display(decoded_matrix)
    _, loss = ngram_fitness(decoded_matrix, n=3)
    logger.debug(f"Loss: {loss:.3f}, best: {best_loss:.3f}")       
    if not best_keys or loss < best_loss:
        logger.debug(f"[SUCCESS GREAT!] New best key: {key} ({loss})")
        best_loss = loss
        best_keys = [key]
    elif np.equal(loss, best_loss):
        logger.debug(f"[SUCCESS!] Adding best key: {key}")
        best_keys.append(key)
    if idx == 0 or ((idx+1) % int(0.05*max_iters) == 0):
        logger.info(f"[INFO] [{idx+1}/{max_iters} ({(idx+1)/max_iters*100:.2f}%)] "
                    f"Loss: {best_loss}, num keys: {len(best_keys)}")
        

06:44:08 INFO:[INFO] [1/100000 (0.00%)] Loss: 96.5013102746617, num keys: 1


KeyboardInterrupt: 

In [212]:
import random

def hill_climbing_round(crypttext, current_key: List[int], current_loss: float, 
                        allowed_positions: List[int]=None, n: int=3, 
                        decode_argname: str="k2", decode_kwargs={}):
    best_loss = current_loss
    max_iters = len(current_key)*(len(current_key)-1)
    if not allowed_positions:
        allowed_positions = range(len(current_key))
    for idx, swap in enumerate(itertools.permutations(allowed_positions, 2)):
        key = swap_elements(current_key, swap=swap)
        logger.debug(f"[{idx+1}/{max_iters}] Trying swapped key {key}...")
        _, loss = ngram_fitness(decode(df, **{decode_argname: key}, **decode_kwargs), n=n)
        logger.debug(f"Loss: {loss:.3f}, best: {best_loss:.3f}")
        if loss < best_loss:
            logger.debug(f"Found better key: {key} with loss {loss:.3f}!")
            return key, loss
    logger.debug(f"Did not found any better key for {current_key}!")
    return None, None

def hill_climbing_phase(crypttext, starting_key: List[int], starting_loss: float=None,
                        allowed_positions=None, n: int=3, 
                        decode_argname: str="k2", decode_kwargs={}):
    if starting_loss is None:
        _, starting_loss = ngram_fitness(decode(crypttext, **{decode_argname: starting_key}, **decode_kwargs), n=n)
    improved_key = current_key = starting_key
    current_loss = starting_loss
    while improved_key is not None:
        logger.debug(f"Starting Hill Climbing round with {current_key}...")
        improved_key, improved_loss = hill_climbing_round(crypttext, current_key, current_loss, 
                                                          allowed_positions=allowed_positions, n=n,
                                                          decode_argname=decode_argname, decode_kwargs=decode_kwargs)
        logger.debug(f"Improved key: {improved_key} ({improved_loss})")
        if improved_key is not None:
            current_key, current_loss = improved_key, improved_loss
    logger.debug(f"[STOP] best key: {current_key}, best loss {current_loss:.3f} "
                 f"(improved? {current_key!=starting_key})")
    return current_key, current_loss

def hill_climbing(crypttext, starting_key: List[int], n: int=3, phases: int=100,
                  link_key_indices=None, permutate: bool=False, 
                  decode_argname: str="k2", decode_kwargs={}):
    _, starting_loss = ngram_fitness(decode(crypttext, **{decode_argname: starting_key}, **decode_kwargs), n=n)
    best_keys, best_loss = {tuple(starting_key)}, starting_loss
    current_key, current_loss = starting_key, starting_loss
    # needed to produce new key
    group_indices = list(range(len(starting_key)))
    if link_key_indices:
        for link_idx, link in enumerate(link_key_indices):
            group_indices = [group_idx for group_idx in group_indices 
                             if starting_key[group_idx] not in link] + [-link_idx-1]
    allowed_positions = [idx for idx in group_indices if idx >= 0]
    for phase in range(phases):
        if (phase+1) % int(0.05*phases) == 0:
            logger.info(f"Phase [{phase+1}/{phases} {100*(phase+1)/phases:.2f}%]...")
        logger.info(f"Allowed positions: {allowed_positions} in key {current_key}")
        new_key, new_loss = hill_climbing_phase(crypttext, current_key, current_loss, 
                                                allowed_positions=allowed_positions, n=n,
                                                decode_argname=decode_argname, decode_kwargs=decode_kwargs)
        if new_loss < best_loss:
            logger.info(f"[GREAT SUCCESS] Best key is updated: {new_key} ({new_loss:.3f})")
            best_keys, best_loss = {tuple(new_key)}, new_loss
        elif np.equal(new_loss, best_loss):
            logger.info(f"[SUCCESS] Best key is added: {new_key} ({new_loss:.3f})")
            best_keys.add(tuple(new_key))
            
        # produce new key
        if not link_key_indices:
            random.shuffle(current_key)
            allowed_positions=None
        else:
            random.shuffle(group_indices)
            # expand group indices
            current_key = []
            allowed_positions = []
            for group_index in group_indices:
                if group_index >= 0:
                    current_key.append(starting_key[group_index])
                    allowed_positions.append(len(current_key)-1)
                else:
                    current_key += [idx for idx in link_key_indices[-group_index-1]]
            
            #logger.info(f"Generated key with linked groups: {current_key}")
        _, current_loss = ngram_fitness(decode(crypttext, k2=current_key), n=n)
        logger.debug(f"Generated new key: {current_key} ({current_loss:.3f})")
    logger.info(f"[END] Keys: {best_keys} Best loss: {best_loss}")
    return list(best_keys), best_loss
        
    
best_keys, best_loss = hill_climbing(df, [9, 4, 8, 0, 1, 3, 5, 6, 7, 2], phases=100, n=4,
                                     link_key_indices=[[9, 4, 8, 0, 1]],
                                     decode_argname="k1", decode_kwargs={"k2": [6, 4, 0, 9, 5, 7, 1, 10, 3, 2, 8,]})

07:14:23 INFO:Allowed positions: [5, 6, 7, 8, 9] in key [9, 4, 8, 0, 1, 3, 5, 6, 7, 2]
07:14:23 INFO:[GREAT SUCCESS] Best key is updated: [9, 4, 8, 0, 1, 2, 6, 3, 7, 5] (98.228)
07:14:23 INFO:Allowed positions: [0, 1, 2, 3, 9] in key [7, 6, 2, 5, 9, 4, 8, 0, 1, 3]
07:14:23 INFO:[GREAT SUCCESS] Best key is updated: [6, 3, 7, 5, 9, 4, 8, 0, 1, 2] (98.228)
07:14:23 INFO:Allowed positions: [0, 1, 7, 8, 9] in key [3, 2, 9, 4, 8, 0, 1, 6, 5, 7]
07:14:23 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [2, 6, 3, 5, 7, 9, 4, 8, 0, 1]
07:14:23 INFO:Phase [5/100 5.00%]...
07:14:23 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [5, 7, 3, 2, 6, 9, 4, 8, 0, 1]
07:14:23 INFO:Allowed positions: [0, 6, 7, 8, 9] in key [7, 9, 4, 8, 0, 1, 2, 5, 6, 3]
07:14:23 INFO:[GREAT SUCCESS] Best key is updated: [6, 9, 4, 8, 0, 1, 2, 3, 7, 5] (98.220)
07:14:23 INFO:Allowed positions: [0, 6, 7, 8, 9] in key [5, 9, 4, 8, 0, 1, 6, 3, 7, 2]
07:14:23 INFO:[SUCCESS] Best key is added: [6, 9, 4, 8, 0, 1, 2, 3, 7, 5] (98.220

07:14:31 INFO:Allowed positions: [5, 6, 7, 8, 9] in key [9, 4, 8, 0, 1, 6, 3, 7, 5, 2]
07:14:31 INFO:Allowed positions: [0, 1, 2, 8, 9] in key [7, 6, 2, 9, 4, 8, 0, 1, 3, 5]
07:14:32 INFO:Phase [80/100 80.00%]...
07:14:32 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [3, 7, 2, 6, 5, 9, 4, 8, 0, 1]
07:14:32 INFO:Allowed positions: [0, 1, 7, 8, 9] in key [7, 5, 9, 4, 8, 0, 1, 6, 2, 3]
07:14:32 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [7, 3, 2, 5, 6, 9, 4, 8, 0, 1]
07:14:32 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [7, 3, 6, 2, 5, 9, 4, 8, 0, 1]
07:14:32 INFO:Allowed positions: [0, 1, 2, 3, 9] in key [6, 7, 5, 3, 9, 4, 8, 0, 1, 2]
07:14:32 INFO:[SUCCESS] Best key is added: [3, 7, 5, 6, 9, 4, 8, 0, 1, 2] (98.218)
07:14:32 INFO:Phase [85/100 85.00%]...
07:14:32 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [7, 2, 6, 5, 3, 9, 4, 8, 0, 1]
07:14:32 INFO:Allowed positions: [0, 1, 2, 3, 4] in key [5, 7, 2, 6, 3, 9, 4, 8, 0, 1]
07:14:32 INFO:Allowed positions: [5, 6, 7, 8, 9] in key 

In [169]:
best_keys[:10]

[(6, 4, 0, 9, 8, 7, 1, 5, 3, 2)]

In [170]:
old_keys_k2 = [(5, 7, 1, 2, 10, 3, 8, 6, 4, 0, 9), [9, 5, 7, 1, 10, 3, 2, 8, 6, 4, 0], [4, 6, 2, 8, 9, 5, 7, 1, 10, 3, 0]]
old_keys_k1 = [[6, 4, 0, 9, 8, 7, 1, 5, 3, 2], [6, 4, 2, 8, 7, 3, 1, 5, 0, 9]]

In [220]:
decoded = decode(df, k1=(9, 4, 8, 0, 1, 3, 7, 5, 6, 2), k2=[6, 4, 0, 9, 5, 7, 1, 10, 3, 2, 8,])
display(decoded)
ngram_fitness(decoded, n=4)
''.join(from_df(decoded))

,6,4,0,9,5,7,1,10,3,2,8
9,A,R,T,H,U,R,I,J,U,S,T
4,H,A,D,T,H,E,S,T,R,A,N
8,G,E,S,T,D,R,E,A,M,I,W
0,A,S,T,A,K,I,N,G,S,O,M
1,E,M,A,T,H,T,E,S,T,I,H
3,A,D,N,T,S,T,U,D,I,E,D
7,F,O,R,A,N,D,T,H,E,N,Y
5,O,U,T,R,I,E,D,T,O,S,A
6,W,O,F,F,M,Y,H,E,A,D,W
2,E,I,R,D,H,U,H,-,-,-,-


'ARTHURIJUSTHADTHESTRANGESTDREAMIWASTAKINGSOMEMATHTESTIHADNTSTUDIEDFORANDTHENYOUTRIEDTOSAWOFFMYHEADWEIRDHUH----'

In [ ]:
[9, 4, 8, 0, 1, 3, 7, 5, 6, 2]